# Test that file handler can download things and use the downloaded copy before downloading again

In [1]:
import sys
sys.path.append(sys.path[0]+'/../')

import helpers.fileHandler as fh

In [ ]:
#something that works
fh.awsDownloader('ACCESS-CM2', 'ps_Amon', 'historical')

In [ ]:
#something that doesnt work?
fh.awsDownloader('ACCESS-CM2', 'psl_Amon', 'historical')

In [ ]:
fh.loadModelData('ACCESS-CM2', 'psl_Amon', 'ssp126')

In [ ]:
fh.loadModelData('ACCESS-CM2', 'ps_Amon', 'historical')

In [ ]:
fh.awsDownloader('ACCESS-CM2', 'areacella_fx', 'ssp370')

In [ ]:
'assp370'.find('ssp')

In [2]:
fh.esgfDownloader('AWI-CM-1-1-MR', 'ts_Amon', 'ssp585', 'r1i1p1f1')

KeyboardInterrupt: 

In [3]:
fh.esgfDownloader('AWI-CM-1-1-MR', 'psl_Amon', 'ssp585', 'r1i1p1f1')

CMIP6.ScenarioMIP.AWI.AWI-CM-1-1-MR.ssp585.r1i1p1f1.Amon.psl.gn.v20190529|esgf3.dkrz.de downloading


KeyboardInterrupt: 

In [7]:
fh.esgfDownloader('CAS-ESM2-0', 'tas', 'ssp585', 'r1i1p1f1')

file not found on ESGF


False

In [6]:
from pyesgf.logon import LogonManager
lm = LogonManager()
lm.logoff()
lm.is_logged_on()

False

In [9]:
myHost='esgf.nci.org.au'
lm.logon(username='maelgwn', 
         password='Jm4*62%FCA2h', hostname=myHost)

In [8]:
from pyesgf.search import SearchConnection

#conn = SearchConnection('https://esgf-node.llnl.gov/esg-search')
conn = SearchConnection('https://esgf-data.dkrz.de/esg-search')
#conn = SearchConnection('https://esgf.nci.org.au/esg-search')

ctx = conn.new_context(
    project='CMIP6', 
    source_id='UKESM1-0-LL', 
    experiment_id='historical', 
    variable='tas', 
    frequency='mon', 
    variant_label='r1i1p1f2',
   #data_node='esgf-data3.ceda.ac.uk'
)
ctx.hit_count

result = ctx.search()
result[0].dataset_id

'CMIP6.CMIP.MOHC.UKESM1-0-LL.historical.r1i1p1f2.Amon.tas.gn.v20190406|esgf3.dkrz.de'

In [6]:
result[0].filename()


AttributeError: 'DatasetResult' object has no attribute 'filename'

In [14]:
#%%writefile dl.sh
with open('dl.sh', "w") as writer:
    writer.write(
        result[0].file_context().get_download_script()
    )

import subprocess
import os

os.chmod('dl.sh', 0o750)
subprocess.check_output('bash ./dl.sh -s', shell=True)